# Distributed Databases and Big Data


**Instructions:**
- You will be using Python 3.
- Read the assignment instruction carefully and implement the algorithms in this workbook. 
- You can use the datasets fireData and climateData (provided below) if you are aiming for Credit Task.
- For Distinction and High Distinction tasks, you are required to read the files FireData.csv and ClimateData.CSV provided with the assignment programatically and prepare the data in the correct format so that it can be used in your algorithm. 
- You can introduce new cells as necessary.

**Your details**
- Name: FANCHAO KONG 



Let's get started!

In [29]:
#import pandas to read file
import csv
import multiprocessing as mp
from multiprocessing import Pool
import datetime
import time

In [30]:
# read file data to memeory
# fireData
fireData = []
with open('FireData.csv') as firePath:
    reader = csv.reader(firePath)
    for row in reader:
        fireData.append(row)
#climateData
climateData = []
with open('ClimateData.csv') as climatePath:
    reader = csv.reader(climatePath)
    for row in reader:
        climateData.append(row)
del fireData[0]
del climateData[0]

In [31]:
climateData[:1]

[['948700',
  '2016-12-31',
  '19',
  '56.8',
  '7.9',
  '11.1',
  '   72.0*',
  '  61.9*',
  ' 0.00I']]

In [32]:
fireData[:1]

[['-37.966',
  '145.051',
  '341.8',
  '2017-12-27T04:16:51',
  '26.7',
  '78',
  '2017-12-27',
  '68']]

## Task 1 ：Parallel Search

### 1. Write an algorithm to search climate data for the records on  15th December 2017 . Justify your choice of the data partition technique and search technique you have used.


In [33]:
'''
Assume that we have 3 processors, as a result we can divde the data to 3 parts by using two time nodes. 
For divde equally, the time node shuold be the date of 10 and 20. If date less than 10 put it to first processor, else 
if it greater than 10 and less 20, put it to second processor, else put it to the last one.
'''
def task1_1_whichProcessor(date):

    if int(date[-2:]) <= 10:
        return 0
    else:
        if int(date[-2:]) <= 20:
            return 1
        else:
            return 2
        
'''
This is the algorithm how we partioning data, for each record in the target table, we take the date value and compare
with the time node. This function will back a value to piont the exactly which processor
'''

def task1_1_partionByTime(table):
    result = []
    
    for i in range(3):
        result.append([])
        

    for eachRecord in table:
        result[task1_1_whichProcessor(eachRecord[1])].append(eachRecord)
    return result


'''
We alread get a partioned data. When user input a date value, firstly we use "task1_1_whichProcessor" to determine
which worker has may have this value, then searh one by one in that worker. Everytime we find the value we put it to
result list.
'''

def serialSearh(table, date):
    for eachRecord in table:
        if eachRecord[1] == date:
            return eachRecord
def task1_1_searching(date, partionedTable):
    result = []
    workerTask = []
    pool = Pool(processes= 3)
    
    for i in partionedTable:
        workerTask = pool.apply_async(serialSearh, args=(i,date))
        output = workerTask.get()
        result.append(output)
    
    pool.close()
    return result

In [34]:
A = task1_1_partionByTime(climateData)

In [35]:
task1_1_searching('2017-12-15',task1_1_partionByTime(climateData))

[None,
 ['948702',
  '2017-12-15',
  '18',
  '52',
  '7.1',
  '14',
  '   74.5*',
  '53.1',
  ' 0.00I'],
 None]

### 2. Write an algorithm to find the latitude, longitude and confidence when the surface temperature (°C) was between 65 °C and 100 °C. Justify your choice of the data partition technique and search technique you have used.

In [36]:
# linear search by range
'''linear search: more flex in this time.
Use binary search is inefficient (cause longer processing time) '''
def linear_range_search(data, temperary_range):
    result = []
    found = None
    for i in data:
        temp = int(i[-1])
        if (temp >= int(temperary_range[0])) and (temp <= int(temperary_range[1])):
            result.append(i)
    return result

def index_calculate(data):
    index = 0
    if data > 100:
        index = 10
    elif data < 0:
        index = 0
    else:
        index = data/10
    return index   

In [37]:
# range_partitioning according to the temperature
'''the surface temperature in fire data is not sorted therefore use range partitioning'''
def Task1_2_searching(data, temperature_indices):
    results = []
    # range_partitioning
    subsets = []
    for i in range(11):
        subsets.append([])         
    for row in data[1:]:
        index = int(index_calculate(int(row[-1])))
        subsets[index].append(row)
        
    # parallel search
    minIndex = int(index_calculate(temperature_indices[0]))
    maxIndex = int(index_calculate(temperature_indices[1]))
    newData = data[minIndex:(maxIndex+1)]
    
    pool = Pool(processes=3)
    
    for i in newData:
        result = pool.apply(linear_range_search,[data, temperature_indices])
        results.append(result)
    pool.close()
        
    return result

In [38]:
Task1_2_searching(fireData,[65,100])[:1]

[['-37.966',
  '145.051',
  '341.8',
  '2017-12-27T04:16:51',
  '26.7',
  '78',
  '2017-12-27',
  '68']]

## Task2：Parallel Join
### 1. Write an algorithm to find surface temperature (°C), air temperature (°C), relative humidity and maximum wind speed. Justify your choice of the data partition technique and join technique you have used.
    

In [39]:
def month(climate_date):
    return int(climate_date[5:7])#month in ClimitData date 
'''range partition: 
    divide equally into [January to April],[May to August],[September to December] 
'''
def range_partition(data, column):
    result = [[],[],[]] 
    for i in data:
        if month(i[column])<=4:#month smaller than Apr
            result[0].append(i)
        elif month(i[column])>8:
            result[2].append(i)
        else:
            result[1].append(i)
    return result

In [40]:
'''sort merge join:
    sort the two tables (firedata climate data) based on the join attribute date
    then merge these two tables. 
    Then do R and S comparsion. When the two date value match, palce the calue into query result.
'''
def sort_merge_join(data1,column_num1,attr1,data2, column_num2,attr2): 
    result = []
    s_T1 = data1
    s_T1 = sorted(s_T1, key=lambda s_T1: s_T1[column_num1]) #Climate data date col
    s_T2 = data2
    s_T2 = sorted(s_T2, key=lambda s_T2: s_T2[column_num2])#fire data date col
    i = j = 0
    while True:
        r = s_T1[i][column_num1]
        s = s_T2[j][column_num2]
        if r < s: #date r< date s 
            i += 1
        else:
            if r > s:
                j += 1
            else:
                record=[s_T1[i][column_num1]]
                for x in attr1:
                    record.append(s_T1[i][x])
                for y in attr2:
                    record.append(s_T2[j][y])
                result.append(record)
                j += 1

        if (i == len(s_T1)) or (j == len(s_T2)):
            break
    return result

In [41]:
def task2_1(data1,column_num1,attr1,data2,column_num2,attr2):
    results = []
    pool = mp.Pool(processes=3)
    T1_subsets = range_partition(data1, column_num1)
    T2_subsets = range_partition(data2, column_num2)
    for i in range(3):
        #apply sort merge join
        result = pool.apply(sort_merge_join, [T1_subsets[i],column_num1,attr1,T2_subsets[i], column_num2,attr2])
        results.append(result)
    
    for i in results:
        print(len(i))
    return results
    pool.close()

In [44]:
Climate_attr = [2,3,4]#air temp, relatively humidy,windspeed
Fire_attr = [7]#surface temp

task2_1(climateData,1,Climate_attr,fireData,6,Fire_attr)[1][0:5]

1613
781
274


[['2017-05-01', '14', '49.2', '10.5', '50'],
 ['2017-05-01', '14', '49.2', '10.5', '103'],
 ['2017-05-01', '14', '49.2', '10.5', '52'],
 ['2017-05-01', '14', '49.2', '10.5', '39'],
 ['2017-05-01', '14', '49.2', '10.5', '41']]

### 2. Write an algorithm to find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100. Justify your choice of the data partition technique and join technique you have used.

In [45]:
'''
For this task, I use date to partion data. 
The reason why I use date to partion is that using it can distribute task basically equally for each worker. 
'''
def task2_2_whichProcessor(date):

    if int(date[-2:]) <= 10:
        return 0
    else:
        if int(date[-2:]) <= 20:
            return 1
        else:
            return 2

In [46]:
'''
for this task, the date in each table is sorted by date, so I perfer to use sort-merge join.
for each date both in firedata and climate. We can join them by using date easily.
'''
def task2_2_partionByTime(climateData, fireData):
    result = []
    fireData = sorted(fireData, key= lambda fireData:fireData[-2])
    for i in range(3):
        result.append([])
        for j in range(2):
            result[i].append([])
        

    for eachRecord in climateData:
        result[task2_2_whichProcessor(eachRecord[1])][0].append(eachRecord)
        
    for eachRecord in fireData:
        result[task2_2_whichProcessor(eachRecord[-2])][1].append(eachRecord)
    
    for eachworker in result:
        for eachList in eachworker:
            eachList.append(["This","is","the","end"])
    return result

In [47]:
'''
for each worker, they has two sorted table. One is part of firedata, other is part of climate.
Using sort-merge in this task can decrease the caculate time.
'''
def task2InnerJoin(table):
    result = []
    fireList = list (table[1])
    climateList = list(table[0])
    climateIndex = fireIndex = 0
    while True:
        climateDate = climateList[climateIndex][1]
        fireDate = fireList[fireIndex][-2]

        if climateDate > fireDate:
            fireIndex = fireIndex + 1
        elif climateDate < fireDate:
            climateIndex = climateIndex + 1
        else:
            if int(fireList[fireIndex][-3]) >= 80 and int(fireList[fireIndex][-3])<=100:
                result.append({",".join([str(fireList[fireIndex][3]),str(fireList[fireIndex][5]),str(fireList[fireIndex][-1]),str(climateList[climateIndex][2])])})
            if climateList[climateIndex][1] == fireList[fireIndex+1][-2]:
                fireIndex += 1
            else:
                fireIndex += 1
                climateIndex += 1
        if (climateIndex >= len(climateList)-1) or (fireIndex >= len(fireList)-1):
            break
    return result

def Task2_2(partionedTable):
    result = []
    local_result = []
    workerTask = []
    pool = Pool(processes= 3)
    
    for i in partionedTable:
        workerTask = pool.apply_async(task2InnerJoin, [i])
        output = workerTask.get()
        for eachJoining in output:
            result.append(eachJoining)
    pool.close()
    
    return result

In [48]:
task2InnerJoin(task2_2_partionByTime(climateData, fireData)[1])[0:5]

[{'2017-03-12T04:33:50,94,105,21'},
 {'2017-03-12T04:29:50,84,71,21'},
 {'2017-03-12T04:28:20,100,99,21'},
 {'2017-03-12T04:28:00,80,68,21'},
 {'2017-03-12T04:27:20,85,98,21'}]

In [49]:
len(Task2_2(task2_2_partionByTime(climateData, fireData)))

1133

## Task 3 Sort
### 1.Write an algorithm to sort fire data based on surface temperature (°C) in a ascending order. Justify your choice of the data partition technique and sorting technique you have used.

In [50]:
'''
In this task, the main point is surface temperature. So we can not use date to partion. Otherwhise the unmber of task 
for each worker is very unbalance. In other hand, we do not know too much about the value of surface temperature. As 
result, using round-robin is go way to dual with these data.
'''
def task3DataPartion(fireData):
    result = []
    for i in range(3):
        result.append([])
    
    for index, eachRecord in enumerate(fireData):
        index_bin = int(index%3)
        result[index_bin].append(eachRecord)
    
    return result

In [51]:
'''
quick sort is a very effective method to sort data
'''
def quicksort(table):
    if len(table) <= 1:
        return table
    less = []
    greater = []
    baseRecord = table.pop()
    base = baseRecord[2]
    for x in table:
        if x[2] < base:
            less.append(x)
        else:
            greater.append(x)
    return quicksort(less) + [baseRecord] + quicksort(greater)


In [52]:
'''
Bucause we assume that we have three worker, so we using 3-way merge. each way of a result made by each worker.
'''
def findMin(table):
    minIndex = 0
    minNum = float(table[0][2])
    for index in range(len(table)):
        if float(table[index][2]) < minNum:
            minNum = float(table[index][2])
            minIndex = index
    return minIndex

def ThreeWay(SortdePartionedTable):
    indexes = []
    result = []
    T = []
    ifMax = ['max','max','1000000000']
    for i in range(len(SortdePartionedTable)):
        indexes.append(0)
    
    while(True):
        T = []
        for i in range(len(SortdePartionedTable)):
            
            if (indexes[i]>= len(SortdePartionedTable[i])):
                T.append(ifMax)
            else:
                T.append(SortdePartionedTable[i][indexes[i]])
        #print(T)
        
        smallest = findMin(T)
        smallestRecord = T[smallest]
    
        if(T[smallest] == ifMax):
            break
        
        result.append(smallestRecord)
        indexes[smallest] += 1
    return result
    

In [53]:
def paralleSort(partionedTable):
    result = []
    pool = Pool(processes=3)
    
    for eachList in partionedTable:
        workTask = pool.apply_async(quicksort, [eachList])
        result.append(workTask.get())
    pool.close()
    return result

In [54]:
ThreeWay(paralleSort(task3DataPartion(fireData)))[:1]

[['-37.886',
  '147.207',
  '302',
  '2017-07-02T04:28:42',
  '10.7',
  '50',
  '2017-07-02',
  '28']]

## Task 4: Parallel Group-By
### 1. Write an algorithm to get the number of fire in each day. You are required to only display total number of fire and the date in the output. Justify your choice of the data partition technique if any.

In [55]:
'''
Round-Robin is for balance task for each worker. then do inner group by. get each result of worker, put them to a dic
the key of dic is the date, the value represent the how many fire in that day.
'''
def task4DataPartion(table):
    result = []
    #asumme that we have 3 workers
    for i in range(3):
        result.append([])
    #for the balance work of each processor, I choose round roll bin partion
    for index, eachrecord in enumerate(table):
        index_bin = index%3
        result[index_bin].append(eachrecord)   
    return result

def innerGroupBy(table):
    dic = {}
    for eachRecord in table:
        key = eachRecord[-2]
        if key not in dic:
            dic[key] = 1
        else:
            dic[key] += 1
    return dic

def parallelMergeGroupBy(table):
    result = {}
    pool = Pool(processes=3)
    
    local_result_list = []
    for i in table:
        local_result = pool.apply_async(innerGroupBy, [i])
        output = local_result.get()
        local_result_list.append(output)
    pool.close()

    for dic in local_result_list:
        for eachDate in dic:
            if eachDate not in result:
                result[eachDate] = dic.get(eachDate)
            else:
                result[eachDate] += dic.get(eachDate)  
    return result

In [58]:
parallelMergeGroupBy(task4DataPartion(fireData))

{'2017-12-27': 4,
 '2017-12-21': 1,
 '2017-12-16': 15,
 '2017-12-15': 4,
 '2017-12-13': 1,
 '2017-12-10': 2,
 '2017-12-09': 4,
 '2017-12-08': 5,
 '2017-11-30': 31,
 '2017-11-29': 8,
 '2017-11-23': 5,
 '2017-11-21': 1,
 '2017-11-14': 3,
 '2017-11-13': 5,
 '2017-11-12': 5,
 '2017-11-11': 4,
 '2017-11-09': 10,
 '2017-11-05': 4,
 '2017-10-27': 5,
 '2017-10-26': 5,
 '2017-10-23': 1,
 '2017-10-21': 4,
 '2017-10-20': 3,
 '2017-10-18': 6,
 '2017-10-17': 5,
 '2017-10-15': 3,
 '2017-10-10': 3,
 '2017-10-08': 1,
 '2017-10-06': 2,
 '2017-10-04': 5,
 '2017-10-03': 18,
 '2017-10-02': 7,
 '2017-10-01': 8,
 '2017-09-29': 2,
 '2017-09-27': 7,
 '2017-09-26': 1,
 '2017-09-24': 28,
 '2017-09-23': 23,
 '2017-09-21': 2,
 '2017-09-20': 5,
 '2017-09-10': 4,
 '2017-08-14': 5,
 '2017-08-13': 9,
 '2017-08-10': 1,
 '2017-08-02': 2,
 '2017-07-31': 2,
 '2017-07-29': 2,
 '2017-07-06': 3,
 '2017-07-05': 1,
 '2017-07-02': 8,
 '2017-07-01': 4,
 '2017-06-30': 6,
 '2017-06-20': 6,
 '2017-06-18': 2,
 '2017-06-16': 2,
 '20

In [59]:
A = parallelMergeGroupBy(task4DataPartion(fireData))
x = 0
for i in A:
    x += A.get(i)
print(x)

2668


### 2.Write an algorithm to find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) ​and the date in the output. Justify your choice of the data partition technique if any.




In [60]:
'''Round-Robin: simply and equally divide each record into each processor'''
def task4DataPartion2(data):
    result = []
    for i in range(3):
        result.append([])
    
    for index, element in enumerate(data): 
        index_bin = index%3
        result[index_bin].append(element)
    return result

In [61]:
def task4_groupby(dataset, date, surface):
    dict = {}
    for index, record in enumerate(dataset):
        key = record[date]
        val = int(record[surface])
        if key not in dict:
            dict[key] = [0,0]
        dict[key][0] += val
        dict[key][1] += 1
    return dict

In [62]:
'''merge_all_groupby has two phases: a local aggregation step and global aggregation step
-local groupby: groups local records according to date attribute and performs the aggregation function.
-global groupby: final aggregation in each processor
Finally 
-Calculate average temperature'''
def task4_parallel_merge_all_groupby(dataset):
    subsets = task4DataPartion2(dataset)
    pool = mp.Pool(processes = 3)

    # Local aggregation step
    local_result = []
    for s in subsets:
        local_result.append(pool.apply(task4_groupby, [s, 6, 7])) 
    pool.close()

    # Global aggregation step
    result = {}
    for r in local_result:
        for key, val in r.items():
            if key not in result:
                result[key]= [0,0]
            result[key][0] += val[0]
            result[key][1] += val[1]
            
    #Average temperate
    average = {} 
    for key, val in result.items():
         average[key] = round(float(val[0]/val[1]),2)
    return average

In [63]:
task4_parallel_merge_all_groupby(fireData)

{'2017-12-27': 62.75,
 '2017-12-21': 46.0,
 '2017-12-16': 57.8,
 '2017-12-15': 39.0,
 '2017-12-13': 60.0,
 '2017-12-10': 46.0,
 '2017-12-09': 58.25,
 '2017-12-08': 50.6,
 '2017-11-30': 52.42,
 '2017-11-29': 60.62,
 '2017-11-23': 58.8,
 '2017-11-21': 59.0,
 '2017-11-14': 52.0,
 '2017-11-13': 47.0,
 '2017-11-12': 53.0,
 '2017-11-11': 46.25,
 '2017-11-09': 61.3,
 '2017-11-05': 58.5,
 '2017-10-27': 50.4,
 '2017-10-26': 44.6,
 '2017-10-23': 38.0,
 '2017-10-21': 51.25,
 '2017-10-20': 50.0,
 '2017-10-18': 52.17,
 '2017-10-17': 51.6,
 '2017-10-15': 72.67,
 '2017-10-10': 53.33,
 '2017-10-08': 41.0,
 '2017-10-06': 44.0,
 '2017-10-04': 49.0,
 '2017-10-03': 50.0,
 '2017-10-02': 43.57,
 '2017-10-01': 48.25,
 '2017-09-29': 43.0,
 '2017-09-27': 49.71,
 '2017-09-26': 33.0,
 '2017-09-24': 53.57,
 '2017-09-23': 52.74,
 '2017-09-21': 40.5,
 '2017-09-20': 63.6,
 '2017-09-10': 56.5,
 '2017-08-14': 40.8,
 '2017-08-13': 49.0,
 '2017-08-10': 63.0,
 '2017-08-02': 63.5,
 '2017-07-31': 47.0,
 '2017-07-29': 48.0,

# Part 5
Write an algorithm to find the  average surface temperature   (°C)  for each weather station.You are required to only display  average surface temperature (°C)   and  the station  in the output.Justify your choice of the data partition and join technique.
Hint: You need to join using the date and group by based on station.

In [64]:
'''
In order to distribute task balancely, and the data join by using date. So I choose the hash partion. Assume that we 
have 3 workers. For each worker, I distribute 1/3 of data. And for this task, task is based on date. So I also use the 
sort-merge to join them. After Join, only get the surface tempreture. Get the totoal number of tempreture, then divid 
the number of join result. Put them to a dic, which is the final result.
'''
def task5whichProcessor(date):

    if int(date[-2:]) <= 10:
        return 0
    else:
        if int(date[-2:]) <= 20:
            return 1
        else:
            return 2
def task5_partionByTime(climateData, fireData):
    result = []
    fireData = sorted(fireData, key= lambda fireData:fireData[-2])
    for i in range(3):
        result.append([])
        for j in range(2):
            result[i].append([])
        

    for eachRecord in climateData:
        result[task5whichProcessor(eachRecord[1])][0].append(eachRecord)
        
    for eachRecord in fireData:
        result[task5whichProcessor(eachRecord[-2])][1].append(eachRecord)
    
    for eachworker in result:
        for eachList in eachworker:
            eachList.append(["This","is","the","end"])
    return result


In [65]:
def task5InnerProcess(table):
    result = {}
    fireList = list (table[1])
    climateList = list(table[0])
    climateIndex = fireIndex = 0
    while True:
        climateDate = climateList[climateIndex][1]
        fireDate = fireList[fireIndex][-2]

        if climateDate > fireDate:
            fireIndex = fireIndex + 1
        elif climateDate < fireDate:
            climateIndex = climateIndex + 1
        else:
            key = climateList[climateIndex][0]
            if key in result:
                result[key][0] += float(fireList[fireIndex][-1])
                result[key][1] += 1
            else:
                result[key] = [float(fireList[fireIndex][-1]),1]
                
            if climateList[climateIndex][1] == fireList[fireIndex+1][-2]:
                fireIndex += 1
            else:
                fireIndex += 1
                climateIndex += 1
        if (climateIndex >= len(climateList)-1) or (fireIndex >= len(fireList)-1):
            break
    return result

In [66]:
def task5ParallelProcess(table):
    result = {}
    finalResult = {}
    pool = Pool(processes=3)
    
    for i in table:
        localResult = pool.apply_async(task5InnerProcess, [i])
        localOutput = localResult.get()
        for x in localOutput:
            if x in result:
                result[x][0] += localOutput[x][0]
                result[x][1] += localOutput[x][1]
            else:
                result[x] = localOutput[x]
    pool.close()
    for eachResult in result:
        finalResult[eachResult] = (result[eachResult][0]/result[eachResult][1])
    
    return finalResult

In [67]:
task5ParallelProcess(task5_partionByTime(climateData, fireData))

{'948701': 56.06938603868797, '948702': 52.148275862068964}